In [1]:
from tableauhyperapi import HyperProcess, Connection, TableDefinition, SqlType, Telemetry, Inserter, CreateMode, TableName
from tableauhyperapi import escape_string_literal
from tableau_api_lib.utils.querying import get_projects_dataframe
from tableau_api_lib import TableauServerConnection
import pandas as pd
import time

data = time.strftime("%d%m%Y%H%M")
nome_arquivo = 'BaseT{}.csv'.format(data)
nome = 'Base_{}'.format(data)

#versão da API que você usar no servidor-alvo, em server: link do servidor, se preferir, criar um JSON e importar as cred depois
tableau_server_config = {
        'tableau_prod': {
                'server': '',  
                'api_version': '3.13', 
                'username': 'nome de usuario',
                'password': 'senha',
                'site_name': '',
                'site_url': ''
        }
}
# nome do arquivo ou variavel que contem as credenciais de acesso, ENV = nome do campo que contem as credenciais dentro do arquivo
conn = TableauServerConnection(tableau_server_config, env='tableau_prod')

conn.sign_in()

projects_df = get_projects_dataframe(conn)#Ao rodar o código em um ambiente e dar um "project_df" vai aparecer um dataframe com codigo id dos projetos e proprietarios

default_project_id = 'COLOQUE O PROJECT ID com o GET anterior'

def saveHyper(df):
        df.to_csv(nome_arquivo, index=False)
        PATH_TO_CSV = nome_arquivo
        PATH_TO_HYPER = "Example.hyper"
        my_df = pd.read_csv(PATH_TO_CSV)
        #my_df['date'] = my_df['date'].apply(pd.to_datetime)  


        # Step 1: Start a new private local Hyper instance
        with HyperProcess(Telemetry.SEND_USAGE_DATA_TO_TABLEAU, 'myapp' ) as hyper:

        # Step 2:  Create the the .hyper file, replace it if it already exists
            with Connection(endpoint=hyper.endpoint, 
                            create_mode=CreateMode.CREATE_AND_REPLACE,
                            database=PATH_TO_HYPER) as connection:

        # Step 3: Create the schema
                connection.catalog.create_schema('Extract')

        # Step 4: Create the table definition / criar um TableDefinition para cada coluna no DF
                schema = TableDefinition(table_name=TableName('Extract','Extract'),
                    columns=[
                    TableDefinition.Column('COD', SqlType.double()),
                    TableDefinition.Column('NOME', SqlType.text())
                 ])
            
        # Step 5: Create the table in the connection catalog
                connection.catalog.create_table(schema)
            
                with Inserter(connection, schema) as inserter:
                    for index, row in my_df.iterrows():
                        inserter.add_row(row)
                    inserter.execute()
        response = conn.publish_data_source(datasource_file_path="Example.hyper",
                                 datasource_name=nome,
                                 project_id=default_project_id,
                                 datasource_description="Sem o PREP")
        response
        print(response)

In [8]:
df = pd.read_csv('arquivo.csv', sep=';') #importar o arquivo que deseja com o read_csv na variavel df e rodar a função abaixo:

saveHyper(df)